In [1]:
%matplotlib inline
from refer import REFER
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
from random import randint
import json
from PIL import Image

In [2]:
data_root = './data'  # contains refclef, refcoco, refcoco+, refcocog and images
dataset = 'refcoco'
splitBy = 'unc'
refer = REFER(data_root, dataset, splitBy)

dataset = 'refcoco+'
splitBy = 'unc'
refer_plus = REFER(data_root, dataset, splitBy)

dataset = 'refcocog'
splitBy = 'google'
refer_g = REFER(data_root, dataset, splitBy)

loading dataset refcoco into memory...
creating index...
index created.
DONE (t=5.86s)
loading dataset refcoco+ into memory...
creating index...
index created.
DONE (t=6.25s)
loading dataset refcocog into memory...
creating index...
index created.
DONE (t=5.13s)


In [3]:
ld = set(os.listdir('/media/hdd/anoskov/refcoco/MobileVLM/data/finetune_data/coco/train2014'))

ref_ids = refer_g.getRefIds()
cnt = 0
fails = 0
for ref_id in ref_ids:
    img_name = refer.Refs[ref_id]['file_name'][:27] + '.jpg'
    if img_name in ld:
        cnt += 1
    else:
        fails += 1
cnt, fails

(49822, 0)

In [4]:
def cvt_coords(coords):
    x1 = coords[0] 
    x2 = coords[0] + coords[2]
    y1 = coords[1] 
    y2 = coords[1] + coords[3]
    return x1, x2, y1, y2

### Train and Val data

In [64]:
# base
ref_ids_train = refer.getRefIds(split='train')
ref_ids_val = refer.getRefIds(split='val')
ref_ids = ref_ids_train + ref_ids_val

# plus
ref_ids_train = refer_plus.getRefIds(split='train')
ref_ids_val = refer_plus.getRefIds(split='val')
ref_ids_plus = ref_ids_train + ref_ids_val

# google
ref_ids_train = refer_g.getRefIds(split='train')
ref_ids_g = ref_ids_train + ref_ids_val

ref_ids = [ref_ids, ref_ids_plus, ref_ids_g]
refers = [refer, refer_plus, refer_g]


In [65]:
total_annotations = 0

for i, refs in enumerate(ref_ids):
    for j, ref_id in enumerate(refs):
        total_annotations += len(refers[i].Refs[ref_id]['sentences'])
    print("total_annotations: ", total_annotations)
new_dict = [dict() for x in range(total_annotations)]

total_annotations:  131458
total_annotations:  262407
total_annotations:  355142


In [71]:

basic_phrase = ['<image>\nDetect ', ' and say its coordinates.']

cur_idx = 0

for j, refs in enumerate(ref_ids):
    for i, ref_id in enumerate(refs):
        ref = refers[j].Refs[ref_id]
        replics = ref['sentences'] #['sent']
        id = str(ref['file_name'][15:27])
        filename = 'coco/train2014/' + ref['file_name'][:27] + '.jpg'
        filename_for_open = '/media/hdd/anoskov/refcoco/MobileVLM/data/finetune_data/coco/train2014/' + ref['file_name'][:27] + '.jpg'
        im_size = Image.open(filename_for_open).size
        x, y, w, h = refer.getRefBox(ref_id)
        x1, x2, y1, y2 = cvt_coords([x, y, w, h])
        x1 = int(x1 / im_size[0] * 100)
        x2 = int(x2 / im_size[0] * 100)
        y1 = int(y1 / im_size[1] * 100)
        y2 = int(y2 / im_size[1] * 100)
        if (x1 < 0) or (x2 < 0) or (y1 < 0) or (y2 < 0):
            print(x, y, w, h)
            print(x1, x2, y1, y2)
            print(im_size)
            print(ref_id)
            break
        # conversations = [dict() for i in range(2)]
        for it in replics:
            item = it['sent']
            # basic_phrase = basic_prases #basic_prases[(i + j) % len(basic_prases)]
            phrase = basic_phrase[0] + item + basic_phrase[1]
            conversations = [{'from': 'human', 'value': phrase},
                {'from': 'gpt', 'value': f'Coordinates are {x1}, {x2}, {y1}, {y2}.'}]
            new_dict[cur_idx] = {'id': id, 'image': filename, 'conversations': conversations}
            cur_idx += 1

In [72]:
len(new_dict)

355142

In [73]:
new_dict[170000]

{'id': '000000397315',
 'image': 'coco/train2014/COCO_train2014_000000397315.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nDetect the book above margaret atwood and say its coordinates.'},
  {'from': 'gpt', 'value': 'Coordinates are 0, 56, 0, 110.'}]}

In [74]:
with open("refcocogp_in_llava_format_train_val.json", "w") as outfile:
    json.dump(new_dict, outfile)

### Test data

In [5]:
# base
ref_ids_test = refer.getRefIds(split='test')

# plus
ref_ids_test_plus = refer_plus.getRefIds(split='test')

# google
ref_ids_test_g = refer_g.getRefIds(split='val')

ref_ids = [ref_ids_test, ref_ids_test_plus, ref_ids_test_g]
refers = [refer, refer_plus, refer_g]


In [6]:
total_annotations = 0

for i, refs in enumerate(ref_ids):
    for j, ref_id in enumerate(refs):
        total_annotations += len(refers[i].Refs[ref_id]['sentences'])
    print("total_annotations: ", total_annotations)
new_dict = [dict() for x in range(total_annotations)]

total_annotations:  10752
total_annotations:  21367
total_annotations:  30903


In [9]:

basic_phrase = ['<image>\nDetect ', ' and say its coordinates.']
                # ['<image>\nFind ', ' on the image.'],
                # ['<image>\nGive me coordinates of the ', '.'], 
                # ['<image>\nFind ', ' and give its coordinates.'],
                # ['<image>\nChoose bounding box of ', ' on the image.']]

cur_idx = 0

for j, refs in enumerate(ref_ids):
    for i, ref_id in enumerate(refs):
        ref = refers[j].Refs[ref_id]
        replics = ref['sentences'] #['sent']
        id = str(ref['file_name'][15:27])
        filename = 'coco/train2014/' + ref['file_name'][:27] + '.jpg'
        filename_for_open = '/media/hdd/anoskov/refcoco/MobileVLM/data/finetune_data/coco/train2014/' + ref['file_name'][:27] + '.jpg'
        im_size = Image.open(filename_for_open).size
        x, y, w, h = refer.getRefBox(ref_id)
        x1, x2, y1, y2 = cvt_coords([x, y, w, h])
        x1 = int(x1 / im_size[0] * 100)
        x2 = int(x2 / im_size[0] * 100)
        y1 = int(y1 / im_size[1] * 100)
        y2 = int(y2 / im_size[1] * 100)
        if (x1 < 0) or (x2 < 0) or (y1 < 0) or (y2 < 0):
            print(x, y, w, h)
            print(x1, x2, y1, y2)
            print(im_size)
            print(ref_id)
            break
        # conversations = [dict() for i in range(2)]
        for it in replics:
            item = it['sent']
            # basic_phrase = basic_prases #basic_prases[(i + j) % len(basic_prases)]
            phrase = basic_phrase[0] + item + basic_phrase[1]
            conversations = [{'from': 'human', 'value': phrase},
                {'from': 'gpt', 'value': f'Coordinates are {x1}, {x2}, {y1}, {y2}.'}]
            new_dict[cur_idx] = {'id': id, 'image': filename, 'conversations': conversations}
            cur_idx += 1

In [8]:
with open("refcocogp_in_llava_format_test.json", "w") as outfile:
    json.dump(new_dict, outfile)